https://www.tensorflow.org/text/tutorials/text_classification_rnn

In [3]:
# filtering
# embeding
# max_len - radsej mensie

In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from tensorflow.python.client import device_lib
import tensorflow as tf

print(device_lib.list_local_devices())
tf.test.gpu_device_name()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4664223516110115660
xla_global_id: -1
]


''

In [1]:
import numpy as np
import pandas as pd

import pickle

import os

from gensim import models

from keras.utils import to_categorical, plot_model
import matplotlib.pyplot as plt


import tensorflow as tf
print(tf.version.VERSION)

2.11.0


In [2]:
NUM_CATEGORY = 8
BATCH_SIZE = 16
EPOCHS = 20
SAMPLE_SIZE = 10000

VOCAB_SIZE = 1000 # Maximum vocab size.
MAX_LEN = 1000  # Sequence length to pad the outputs to.

In [3]:
from keras.callbacks import Callback
class History(Callback):
    """
    Callback that records events into a `History` object.

    This callback is automatically applied to
    every Keras model. The `History` object
    gets returned by the `fit` method of models.
    """

    def on_train_begin(self, logs=None):
        if not hasattr(self, 'epoch'):
            self.epoch = []
            self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)


class ModelCheckpoint(Callback):

    def __init__(self, freq, directory):
        super().__init__()
        self.freq = freq
        self.directory = directory

    def on_epoch_begin(self, epoch, logs=None):
        if self.freq > 0 and epoch % self.freq == 0:
            self.model.save_pretrained(os.Path(self.directory, str(epoch)))

    def on_train_end(self, logs=None):
        self.model.save_pretrained(self.directory)

In [4]:
# df = pd.read_csv("dataset_final_stem_only_eng.csv")
df = pd.read_csv("3_Preprocessing/dataset_lemmatized.csv")

# df['type'].unique()
# len(df['type'].unique())
print(df.shape)
df.head()

(39567, 7)


,index,name,link,lang_eng,valid_dir,value,type
0,0,Australian National University,http://www.anu.edu.au/,1,1,anu newsroom anu study anu research innovation...,Education
1,1,Monash University,http://www.monash.edu/,1,1,monash university one top university australia...,Education
2,2,University of Queensland,http://www.uq.edu.au/,1,1,university queensland university queensland ch...,Education
3,3,Macquarie University,http://mq.edu.au/,1,1,macquarie university tertiary education curren...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,1,1,unsw sydney one best university australia unsw...,Education


In [5]:
df["type"].unique()

array(['Education', 'Nonprofit', 'Facility', 'Government', 'Company',
       'Healthcare', 'Other', 'Archive'], dtype=object)

In [6]:
df["type"].value_counts()

Company       12217
Nonprofit      7719
Education      6403
Healthcare     4108
Other          3136
Facility       2566
Government     2045
Archive        1373
Name: type, dtype: int64

In [7]:
# sample_df = df[(df["type"] == "Company") | (df["type"] == "Education")].sample(n=SAMPLE_SIZE)
# sample_df = df[(df["type"] == "Company") | (df["type"] == "Education")]
# sample_df = df.sample(SAMPLE_SIZE)
sample_df = df
sample_df.head(5)

,index,name,link,lang_eng,valid_dir,value,type
0,0,Australian National University,http://www.anu.edu.au/,1,1,anu newsroom anu study anu research innovation...,Education
1,1,Monash University,http://www.monash.edu/,1,1,monash university one top university australia...,Education
2,2,University of Queensland,http://www.uq.edu.au/,1,1,university queensland university queensland ch...,Education
3,3,Macquarie University,http://mq.edu.au/,1,1,macquarie university tertiary education curren...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,1,1,unsw sydney one best university australia unsw...,Education


In [8]:
print(sample_df['type'].unique())
print(sample_df.shape)

['Education' 'Nonprofit' 'Facility' 'Government' 'Company' 'Healthcare'
 'Other' 'Archive']
(39567, 7)


# Tensorflow Keras CountVectorizer layer (LSTM)

In [9]:
from sklearn.model_selection import train_test_split

X = sample_df["value"]
y = sample_df["type"]
y = to_categorical(np.asarray(y.factorize()[0]), num_classes=NUM_CATEGORY)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

In [10]:
# Create vectorization layer
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_LEN
)
vectorize_layer.adapt(np.array(X_train).flatten())

In [11]:
vocab = np.array(vectorize_layer.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'public', 'health', 'university', 'research',
       'service', 'foundation', 'news', 'institute', 'college',
       'transitional', 'center', 'care', 'technology', 'information',
       'product', 'new', 'program', 'science'], dtype='<U23')

In [12]:
EMBEDDINGS_LEN = 300

word2vec_path = 'C:\\Users\\Martin\\Downloads\\GoogleNews-vectors-negative300.bin.gz'
keyed_vectors = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array

In [13]:
# LSTM model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,), dtype=tf.string),
        vectorize_layer,
        
        tf.keras.layers.Embedding(
            input_dim=weights.shape[0],
            output_dim=weights.shape[1],
            weights=[weights],
            trainable=False,
        ),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        
        tf.keras.layers.Dropout(0.25),
        
        tf.keras.layers.Dense(32, activation='relu'),
        
        tf.keras.layers.Dense(8, activation="softmax")
    ])

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  # metrics=[tf.keras.metrics.CategoricalCrossentropy()])
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])

    return model


# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(1,), dtype=tf.string),
#     vectorize_layer,
#     tf.keras.layers.Embedding(
#         input_dim=len(vectorize_layer.get_vocabulary()),
#         output_dim=64,
#         # Use masking to handle the variable sequence lengths
#         mask_zero=True
#     ),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     # tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(8, activation="softmax")
# ])
# model.summary()

In [14]:

# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#               optimizer=tf.keras.optimizers.Adam(1e-4),
#               # metrics=[tf.keras.metrics.CategoricalCrossentropy()])
#               metrics=[tf.keras.metrics.CategoricalAccuracy()])


In [15]:
# model.load_weights("./model_checkpoints/model.08-1.1122-0.6232/model.08-1.1122-0.6232")

In [16]:
model = create_model()
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

# test_loss, test_acc = model.evaluate(X_test, y_test)
#
# print('Test Loss:', test_loss)
# print('Test Accuracy:', test_acc)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 1000)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 1000, 300)         900000000 
                                                                 
 bidirectional (Bidirectiona  (None, 1000, 256)        439296    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                        

In [17]:
# model = keras.models.load_model("./model_checkpoints/model.02-1.5510-0.4848/model.02-1.5510-0.4848.ckpt")
#
# test_loss, test_acc = model.evaluate(X_test, y_test)
#
# print('Test Loss:', test_loss)
# print('Test Accuracy:', test_acc)

In [18]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# simple early stopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
# checkpoint = ModelCheckpoint("best_model", monitor="val_categorical_accuracy", verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

callbacks_list = [
    # tf.keras.callbacks.ModelCheckpoint(
    #     filepath='./model_checkpoints/model.{epoch:02d}-{val_loss:.4f}-{val_categorical_accuracy:.4f}/model.{epoch:02d}-{val_loss:.4f}-{val_categorical_accuracy:.4f}.ckpt',
    #     save_freq='epoch', verbose=1, monitor='val_loss',
    #     save_weights_only=True, save_best_only=False,
    # ),
    ModelCheckpoint(
        filepath='./model_checkpoints/model.{epoch:02d}-{val_loss:.4f}-{val_categorical_accuracy:.4f}',
        freq=1,
        directory="./model_checkpoints/model.{epoch:02d}-{val_loss:.4f}-{val_categorical_accuracy:.4f}'"
    )
    # ModelCheckpoint("best_model", monitor="val_categorical_accuracy", verbose=1, save_best_only=True, mode='max')
]

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, validation_steps=60, callbacks=[callbacks_list], verbose=1)
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, validation_steps=30, verbose=1,use_multiprocessing=True, workers=5)

Epoch 1/20


C:\Users\Martin\AppData\Roaming\Python\Python39\site-packages\keras\backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 59/928 [>.............................] - ETA: 45:21 - loss: 1.9467 - categorical_accuracy: 0.3083

KeyboardInterrupt: 

# Zatial najlepsie:
    75.23 (rozsireny dataset - keras TextVectorization layer, LSTM)
    73.90 (rozsireny dataset - keras TextVectorization layer, LSTM)
    71.28 (rozsireny dataset - keras TextVectorization layer, LSTM)
    67.52 (rozsireny dataset - sklearn TextVectorization, Conv1D)

In [ ]:
# print(model.history.history)
# with open("./model_checkpoints/model1_history", "wb") as file:
#     pickle.dump(model.history.history, file)

In [ ]:
with open("./model_checkpoints/model1_history", "rb") as file:
    history_test = pickle.load(file)

print(history_test)

In [ ]:
# Saving model
# pickle.dump(model, open("model", 'wb'))

In [ ]:
# Load model
# model = pickle.load(open("model", 'rb'))

In [ ]:
# h = history.history
# h = history_test

# plt.plot(h["loss"], label="loss")
plt.plot(history_test["categorical_accuracy"], label="categorical_accuracy")
plt.plot(history_test["val_categorical_accuracy"], label="val_categorical_accuracy")
# plt.plot([0.5698, 0.6503, 0.6952, 0.7135, 0.7158, 0.7161, 0.7243, 0.7312, 0.7280, 0.7275], label="accuracy")
# plt.plot(h["val_loss"], label="val_loss")
# plt.plot(h["val_loss"], label="val_loss")
plt.title("Heavy model", fontsize=20)
plt.legend()
plt.show()

In [ ]:
plt.plot(h["categorical_accuracy"], label="categorical_accuracy")
plt.plot(h["val_categorical_accuracy"], label="val_categorical_accuracy")
# plt.plot(h["val_loss"], label="val_loss")
plt.title("Acc", fontsize=20)
plt.legend()
plt.show()

In [ ]:
# test_loss, test_acc = model.evaluate(gen_val)
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

labels = df.columns
predictions = model.predict(X_test)
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(11,11))
disp.plot(ax=ax, cmap="Greens")
plt.title("Confusion matrix for individual characters", fontsize=20)
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
plt.xlabel("Predicted", fontsize=14)
plt.ylabel("True", fontsize=14)
plt.show()

In [ ]:
# Doterajsie vysledky

# Cely DS:
#   2 kategorie ("Company" a "Education"):
#       15 epoch:
#           Train Acc: 0.9663   Train Loss: 0.0858
#           Test Acc = 0.9286   Test Loss: 0.2270
#       20 epoch:
#           Train Acc: 0.9671   Train Loss: 0.0783
#           Test Acc = 0.9270   Test Loss: 0.2536
#       50 epoch:
#           Train Acc: 0.9738   Train Loss: 0.0609
#           Test Acc = 0.9167   Test Loss: 0.4088
#
#   8 kategorii:
#       70 epoch:
#           Train Acc: 0.82     Train Loss: ?
#           Test Acc: 0.611     Test Loss: ?
#       100 epoch:
#           Train Acc: 0.8647   Train Loss: 0.3846
#           Test Acc: 0.606     Test Loss: 3.0083
#       130 epoch:
#           Train Acc: 0.8856   Train Loss: 0.3287
#           Test Acc: 0.6014    Test Loss: 3.6702
#       150 epoch:
#           Train Acc: 0.8898   Train Loss: 0.3170
#           Test Acc: 0.5954    Test Loss: 3.7761